# Raw to CSV

Process the raw CSV files from the I-V Curve Tracer to get 1 CSV file with only the relevant columns.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import time
from datetime import datetime 

In [2]:
base_path = Path(r"D:\Research\Data") # Base path for raw data files
output_csv = Path("unfiltered_dataset.csv") # Output CSV file path
log_file = Path("processed_files.txt") # Log file to keep track of processed files
error_log = Path("error_log.txt") # Log file for errors
missing_csvs = Path("missing_csv.txt") # Log missing csv filenames
print_every = 2000 # Print progress every 2000 files

### Identified columns on CSV

In [3]:
base_row1_column_names = [
    'date', 'time_start', 'voltage_0', 'current_0', 'power_0', 'wavelength_0', 'spectralirr_0',
    'modtemp_c', 'modtemp_l', 'cell_v', 'irr_horiz_start', 'irr_incl20_start', 'airtemp',
    'humidity_rel', 'pressure_rel', 'air_density', 'wind_speed_kmh',
    'wind_dir', 'humidity_abs', 'pressure_abs', 'wind_speed_ms', 'irr_east_start',
    'irr_west_start', 'irr_floor_ref_start'
]

full_row1_column_names = base_row1_column_names + ['irr_diffuse_start', 'irr_incl15_start']

base_row2_column_names = [
    'module_name', 'time_end', 'voltage_1', 'current_1',
    'power_1', 'wavelength_1', 'spectralirr_1', 'cell_v_end',
    'irr_horiz_end', 'irr_incl20_end', 'irr_east_end', 'irr_west_end',
    'irr_floor_ref_end']

full_row2_column_names = base_row2_column_names + ['irr_diffuse_end', 'irr_incl15_end']

### Error log

This will log errors into a file with the corresponding exception.

In [4]:
def log_error(filename, message, exception=None):
    if exception:
        full_msg = f"{filename} — {message}: {repr(exception)}"
    else:
        full_msg = f"{filename} — {message}"

    # Print to console
    print("❌", full_msg)

    # Append to log file
    with open(error_log, "a", encoding="utf-8") as err_f:
        err_f.write(full_msg + "\n")

# Processing functions

In [7]:
def process_file(file_path):
    filename = file_path.name

    # --- Robust Read Logic from V3 Script ---
    opts = dict(sep=';', header=None, na_values='--', engine='python')
    try:
        # Try default UTF-8 first
        df = pd.read_csv(file_path, encoding='utf-8', encoding_errors='strict', **opts)
    except UnicodeDecodeError:
        # Fallback to latin1 if UTF-8 fails (handles the \xe3 byte)
        try:
            df = pd.read_csv(file_path, encoding='latin1', encoding_errors='replace', **opts)
        except Exception as e:
            log_error(filename, "Error reading CSV even with fallback", e)
            return None
    except Exception as e:
        log_error(filename, "Error reading CSV", e)
        return None

    # Check number of columns
    num_cols = df.shape[1]
    if num_cols == 26:
        current_row1_names = full_row1_column_names
        current_row2_names = full_row2_column_names
    elif num_cols == 24:
        current_row1_names = base_row1_column_names
        current_row2_names = base_row2_column_names
    else:
        log_error(filename, f"Unexpected column count: {num_cols}", None)
        return None
    
    # Parse Row 1
    try:
        row1_list = df.iloc[0, :].dropna().tolist()
        # Check if dropna produced the correct length
        if len(row1_list) == len(current_row1_names):
            df_row1 = pd.DataFrame([row1_list], columns=current_row1_names)
        else:
            # Fallback for Row 1
            df_row1 = pd.DataFrame([df.iloc[0, :len(current_row1_names)].tolist()], columns=current_row1_names)
        row1 = df_row1.iloc[0]
    except Exception as e:
        log_error(filename, "Error parsing Row 1 metadata", e)
        return None

    # Parse Row 2
    try:
        row2_list = df.iloc[1, :].dropna().tolist()
        # Check if dropna produced the correct length (13 or 15)
        if len(row2_list) == len(current_row2_names):
            df_row2 = pd.DataFrame([row2_list], columns=current_row2_names)
        else:
            # Fallback for Row 2: Explicitly pull indices to handle the gaps
            row2_idxs = [0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 21, 22, 23]
            if num_cols == 26:
                row2_idxs += [24, 25] # Add Diffuse and GTI 15 for 26-col files
            
            df_row2 = pd.DataFrame([df.iloc[1, row2_idxs].tolist()], columns=current_row2_names)
        row2 = df_row2.iloc[0]
    except Exception as e:
        log_error(filename, "Error parsing Row 2 metadata", e)
        return None

    try:
        # Generate timestamp from filename
        name = filename.replace('.csv', '')
        # Split by underscore
        parts = name.split('_')
        # Extract date/time parts (skip module name)
        day = int(parts[1])
        month = int(parts[2])
        year = int(parts[3])
        hour = int(parts[4])
        minute = int(parts[5])
        second_str = parts[6].split(' ')[0]
        second = int(second_str)
        
        timestamp = datetime(year, month, day, hour, minute, second)
        
        module_name = row2["module_name"]
    except Exception as e:
        log_error(filename, "Missing metadata fields", e)
        return None
    
    # Compute Spectral Integral
    try:
        wavelength = df.iloc[:2048, 5].copy() # Columns 5 is wavelength (nm) 
        spectral_irradiance = df.iloc[:2048, 6].copy() # Column 6 is spectral irradiance (W/m2/nm)
        computed_irradiance = np.trapezoid(spectral_irradiance, x=wavelength)
    except Exception as e:
        computed_irradiance = np.nan
        # log_error(filename, "Error computing spectral integral", e)
        # return None

    # Get Irradiances
    try:
        # Start irradiances (from row1)
        irr_horiz_start = row1["irr_horiz_start"]
        irr_incl20_start = row1["irr_incl20_start"]
        irr_incl15_start = row1.get("irr_incl15_start", np.nan)
        irr_east_start = row1["irr_east_start"]
        irr_west_start = row1["irr_west_start"]
        irr_floor_ref_start = row1["irr_floor_ref_start"]
        
        # End irradiances (from row2)
        irr_horiz_end = row2["irr_horiz_end"]
        irr_incl20_end = row2["irr_incl20_end"]
        irr_incl15_end = row2.get("irr_incl15_end", np.nan)
        irr_east_end = row2["irr_east_end"]
        irr_west_end = row2["irr_west_end"]
        irr_floor_ref_end = row2["irr_floor_ref_end"]
    except Exception as e:
        log_error(filename, "Missing irradiance data", e)
        return None

    # Get temperature
    try:
        modtemp_c = row1["modtemp_c"]
        modtemp_l = row1["modtemp_l"]
    except Exception as e:
        log_error(filename, "Error reading module temperatures", e)
        return None

    # I-V & P-V Curve validation
    try: 
        measurements = df.iloc[:, [2, 3, 4]].copy()
        measurements.columns = ['voltage', 'current', 'power']
        # Discard the first values (start from voltage close to 0)
        min_idx = measurements['voltage'].idxmin()
        measurements = measurements.iloc[min_idx:].reset_index(drop=True)

    except Exception as e:
        log_error(filename, "Error processing I-V curve", e)
        return None

    try:
        V = measurements["voltage"]
        I = measurements["current"]
        P = measurements["power"]

        Voc = V.max()
        Pmpp = P.max()
        mpp_idx = P.idxmax()
        Vmpp = V[mpp_idx]
        Impp = I[mpp_idx]
        Imin = I.min()
        
    except Exception as e:
        log_error(filename, "Error computing electrical parameters", e)
        return None

    # Delete tail
    try:
        d = np.mean(np.abs(np.diff(I[:200])))
        diffs_20 = np.abs(np.diff(I[:20]))
        # Compare diffs20 to d
        idxs_cola = [i for i, diff in enumerate(diffs_20) if diff > d]
        if idxs_cola:
            corte_inicial = idxs_cola[-1] + 1
            measurements = measurements.iloc[corte_inicial:].reset_index(drop=True)
    except Exception as e:
        log_error(filename, "Error cleaning up cola noise", e)

    # Find Isc using linear regression on the initial segment (V < Voc/4)
    try:
        # Define the threshold for the initial "flat" segment of the curve
        # Using the Voc previously computed in your function
        v_limit = Voc / 4 
        
        # Select points below the threshold
        mask_ini = (measurements["voltage"] < v_limit)
        
        # Ensure we have enough points (at least 3) to perform a stable regression
        if mask_ini.sum() >= 3:
            vx = measurements.loc[mask_ini, "voltage"].values
            iy = measurements.loc[mask_ini, "current"].values
            
            # Perform Linear Regression: I = a*V + b
            # A becomes a matrix of [Voltage, 1]
            A = np.vstack([vx, np.ones_like(vx)]).T
            a, b = np.linalg.lstsq(A, iy, rcond=None)[0]
            
            # The intercept 'b' is the estimated current at V=0 (Isc)
            Isc = float(b)
        else:
            # Fallback for very sparse curves: take the current closest to V=0
            ordered_meas = measurements.sort_values(by="voltage")
            Isc = float(ordered_meas["current"].iloc[0])
            
    except Exception as e:
        log_error(filename, "Error estimating Isc via linear regression", e)
        return None
    
    # Build final dataframe
    try:
        final_df = pd.DataFrame({
            # Metadata
            'filename': [filename],
            'module_name': [module_name],
            'timestamp': [timestamp],

            # MPP
            'Vmpp': [Vmpp],
            'Impp': [Impp],
            'Pmpp': [Pmpp],

            # Electrical characteristics
            'Voc': [Voc],
            'Imin': [Imin],
            'Isc': [Isc],

            # Irradiance
            'G_spec_int': [computed_irradiance],
            'G_tilt20_start': [irr_incl20_start],
            'G_tilt15_start': [irr_incl15_start],
            'G_horiz_start': [irr_horiz_start],
            'G_east_start': [irr_east_start],
            'G_west_start': [irr_west_start],
            'G_refl_start': [irr_floor_ref_start],

            'G_horiz_end': [irr_horiz_end],
            'G_tilt20_end': [irr_incl20_end],
            'G_tilt15_end': [irr_incl15_end],
            'G_east_end': [irr_east_end],
            'G_west_end': [irr_west_end],
            'G_refl_end': [irr_floor_ref_end],

            # Environmental characteristics
            'module_temperature_center': [modtemp_c],
            'module_temperature_lateral': [modtemp_l],
            'air_temperature': [row1['airtemp']],
            'relative_humidity': [row1['humidity_rel']],
            'air_density': [row1['air_density']],
            'abs_pressure': [row1['pressure_abs']],
            'wind_speed_ms': [row1['wind_speed_ms']],
            'wind_direction': [row1['wind_dir']]
        })
    except Exception as e:
        log_error(filename, "Error creating final dataframe", e)
        return None

    return final_df

In [8]:
# Get all CSV files from 2023-2025
all_files = []
skip_prefixes = ('MS711', 'WS500', 'PV')

for year in range(2022, 2026):  # 2022-2025
    year_files = list(base_path.glob(f"{year}/**/*.csv"))
    # Filter out unwanted prefixes
    year_files = [f for f in year_files if not f.name.startswith(skip_prefixes)]
    all_files.extend(year_files)
    print(f"Found {len(year_files)} files in {year}")

print(f"\nTotal: {len(all_files)} CSV files from 2022-2025 (excluding MS711, WS500, PV)")

# Load already processed files
processed_files = set()
if log_file.exists():
    with open(log_file, 'r') as f:
        processed_files = set(line.strip() for line in f)
    print(f"Already processed: {len(processed_files)} files")

# Filter to only new files
files_to_process = [f for f in all_files if f.name not in processed_files]
total_files = len(files_to_process)

print(f"Files to process: {total_files}")

# Determine if we need header
write_header = not output_csv.exists() or output_csv.stat().st_size == 0

Found 419197 files in 2022
Found 425151 files in 2023
Found 440615 files in 2024
Found 424753 files in 2025

Total: 1709716 CSV files from 2022-2025 (excluding MS711, WS500, PV)
Already processed: 1707840 files
Files to process: 732


# Process Loop

In [9]:
processed_count = 0
skipped_due_to_error = 0
start_time = time.time()

with open(output_csv, 'a', newline='', encoding='utf-8') as output_f:
    for i, file_path in enumerate(files_to_process):  # ✅ Use files_to_process
        filename = file_path.name  # ✅ Just filename for consistency
        
        try:
            result_df = process_file(file_path)
        except Exception as e:
            print(f"Unhandled error on file: {filename}")
            skipped_due_to_error += 1
            continue

        if result_df is None:
            skipped_due_to_error += 1
            continue

        result_df.to_csv(output_f, header=write_header, index=False)
        write_header = False

        with open(log_file, 'a') as log_f:
            log_f.write(filename + '\n')  

        processed_count += 1

        # Progress print
        if processed_count % print_every == 0 or processed_count == 1:
            elapsed = time.time() - start_time
            avg_time = elapsed / processed_count
            remaining = avg_time * (total_files - processed_count)
            print(
                f"[{processed_count}/{total_files}] processed "
                f"({processed_count/total_files * 100:.1f}%) "
                f"— Skipped: {skipped_due_to_error} "
                f"— ETA: {remaining / 60:.1f} min"
            )


# Final summary
print(f"\n✅ Extraction complete. Processed: {processed_count}, Skipped due to error: {skipped_due_to_error}")

❌ CDF1150A1_26_04_2022_13_11_29.csv — Unexpected column count: 18
❌ CDF1150A1_26_04_2022_13_16_29.csv — Unexpected column count: 18
❌ DUSTCS6K270P_26_04_2022_13_10_01.csv — Unexpected column count: 18
❌ DUSTCS6K270P_26_04_2022_13_15_01.csv — Unexpected column count: 18
❌ GSA060_26_04_2022_13_11_07.csv — Unexpected column count: 18
❌ GSA060_26_04_2022_13_16_07.csv — Unexpected column count: 18
❌ LG345N1C_26_04_2022_13_10_45.csv — Unexpected column count: 18
❌ LG345N1C_26_04_2022_13_15_45.csv — Unexpected column count: 18
❌ LG370Q1C_26_04_2022_13_10_23.csv — Unexpected column count: 18
❌ LG370Q1C_26_04_2022_13_15_23.csv — Unexpected column count: 18
❌ _26_04_2022_12_21_51.csv — Unexpected column count: 17
❌ _26_04_2022_12_22_13.csv — Unexpected column count: 17
❌ _26_04_2022_13_11_51.csv — Unexpected column count: 17
❌ _26_04_2022_13_12_13.csv — Unexpected column count: 17
❌ _26_04_2022_13_13_05.csv — Unexpected column count: 17
❌ _10_06_2022___.csv — Unexpected column count: 7
[1/732] p